## [NYCDSA Capstone Project] 
# Women's Softball League Power Ranking Estimate

<br>
Koeun Lim (ke.lim.kang@gmail.com)<br>
Kevin Haghi (kevin.haghi@gmail.com)<br>


# Step 1. Web scraping

---
## Project Description



### Project Outline
- Step 1. Web scraping
- Step 2. EDA
- Step 3. Modeling
- Step 4. 

In [2]:
import pandas as pd
import re
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [3]:
driver = webdriver.Chrome("/usr/local/bin/chromedriver")

# load the first page
url = 'http://stats.ncaa.org/rankings/'
driver.get(url)
time.sleep(1)

select_sport = Select(driver.find_element_by_id('sport'))
select_sport.select_by_visible_text('Softball')


In [4]:
select_season = Select(driver.find_element_by_id('acadyr'))
Seasons = ['2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13']

# Add loop here later
select_season.select_by_visible_text(Seasons[0])

In [5]:
select_div = Select(driver.find_element_by_id('u_div'))
select_div.select_by_visible_text('I')

In [6]:
select_period = Select(driver.find_element_by_id('rp'))

period_box = driver.find_element_by_name('rp')
period_options = [x for x in period_box.find_elements_by_tag_name('option')]
for period_option in period_options:
    period_text = period_option.text
    if 'Final Statistics' in period_text:
        select_period.select_by_visible_text(period_text)

In [13]:
select_team = driver.find_element_by_id('stat_type_T_N')
select_team.click()

In [14]:
stats_box = driver.find_element_by_name('Stats')
stats_options = [x for x in stats_box.find_elements_by_tag_name('option')]
Stats_names = []
for idx_stats, stats_option in enumerate(stats_options):
    Stats_names.append(stats_option.text)

del Stats_names[0]

select_stats = Select(driver.find_element_by_id('Stats'))
select_stats.select_by_visible_text(Stats_names[0])

In [15]:
select_stats = Select(driver.find_element_by_id('stat_seq'))

# add loop here later
select_stats.select_by_visible_text(Stats_names[0])

In [16]:
select_len = Select(driver.find_element_by_name('rankings_table_length'))
select_len.select_by_value('-1')

len_str = driver.find_element_by_id('rankings_table_info').text
num_obs = max([int(s) for s in len_str.split() if s.isdigit()])

In [17]:
# Scrape
soup_level1=BeautifulSoup(driver.page_source, 'lxml')

In [18]:
table1 = soup_level1.find("table", attrs={"id": "rankings_table"})
table_data1 = table1.tbody.find_all("tr") 
table_header = []
for th in table1.find_all("th"):
    table_header.append(th.text.replace('\n', ' ').strip())
table_header

['Rank', 'Team', 'G', 'AB', 'H', 'BA']

In [19]:
table_data = pd.DataFrame(columns = table_header)
t_data = {}
for tr in table1.tbody.find_all("tr"): 
    for idx_td,td in enumerate(tr.find_all("td")): 
        t_data[table_header[idx_td]] = td.text.replace('\n', '').strip()
    table_data = table_data.append(t_data,ignore_index=True)
table_data['Team'] = table_data['Team'].str.replace(r'\(.*\)','')    
table_data = table_data.rename(columns={'Team':'College'})
table_data

/Users/koeunlim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,Rank,College,G,AB,H,BA
0,1,Oklahoma,63,"1,640",564,0.344
1,2,George Washington,62,"1,706",582,0.341
2,3,UCLA,62,"1,685",572,0.339
3,4,Virginia Tech,58,"1,543",515,0.334
4,5,James Madison,61,"1,621",539,0.333
...,...,...,...,...,...,...
290,291,Jackson St.,47,"1,112",229,0.206
291,292,South Carolina St.,36,748,152,0.203
292,293,Saint Peter's,56,"1,346",265,0.197
293,294,Morehead St.,45,"1,001",197,0.197


In [20]:
fname = 'Data/NCAAstats/' + Seasons[0][0:2] + Seasons[0][5:7] + '.csv'
table_data.to_csv(fname,index=False)

In [ ]:
driver.close()